# The Sanky diagrams were generated using google colab.
This is the notebook downloaded from colab. It might not run locally.

## Installs

In [2]:
pip install pySankey


In [3]:
!npm install -g electron@6.1.4 orca --unsafe-perm=true --allow-root
!apt-get install xvfb libgtk2.0-0 libgconf-2-4
!chmod +x /tools/node/bin/orca

npm WARN deprecated request@2.88.2: request has been deprecated, see https://github.com/request/request/issues/3142
npm WARN deprecated har-validator@5.1.5: this library is no longer supported
npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
npm WARN deprecated left-pad@1.3.0: use String.prototype.padStart()
npm WARN deprecated request-promise-native@1.0.9: request-promise-native has been deprecated because it extends the now deprecated request package, see https://github.com/request/request/issues/3142
/tools/node/bin/electron -> /tools/node/lib/node_modules/electron/cli.js
/tools/node/bin/orca -> /tools/node/lib/node_modules/orca/bin/orca.js

> electron@6.1.4 postinstall /tools/node/lib/node_modules/electron
> node install.js

+ electron@6.1.4
+ orca@1.3.1
added 290 packages from 312 contributors in 38.814s
Reading 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports and data

In [5]:
import pandas as pd
import re
import numpy as np
from collections import defaultdict

promises=pd.read_csv("/content/drive/MyDrive/nesy/Final_41 - Promises.csv")
protocol=pd.read_csv("/content/drive/MyDrive/nesy/Final_41 - Protocol.csv")

promises = promises.loc[promises['eliminate'] != 1].reset_index()
protocol = protocol.loc[protocol['eliminate'] != 1].reset_index()


In [6]:
#promises.columns

In [7]:
promises = promises[['citation',
       'leverages natural language structure',
       'leverages relational structure', 'Knowledge representation',
       'logic vs rules', 'implicit vs explicit', 'reasoning',
       'out-of-distribution generalization', 'interpretability',
       'reduced data', 'transferability']]

In [8]:
#protocol.columns

In [9]:
protocol = protocol[['citation','Business Use Case',
       'Technical Application', 'Type of Learning', 'Symbolic terms',
       'Neural terms', 'NeSy Category', 'Kautz category','Datasets','Study Quality']]

In [10]:
df=pd.merge(promises, protocol, on="citation", how='inner')
df=df.reset_index()

In [11]:
#df.columns

In [12]:
neural_sup_terms = ['Stacked Hidden Layers',
                    'Euclidean Space',
                    'Sequence-to-Sequence',
                    'Neuro-Symbolic',
                    'Early Generation',
                    'Graphical Models',
                    'Memory Networks',
                    'Linear Models',
                    'Hybrid Models']
bool_columns =    ['leverages natural language structure','leverages relational structure','No Structure Leverage',
                   'reasoning', 'out-of-distribution generalization', 'interpretability','reduced data', 'transferability','none']
columns =         ['NeSy Category','Kautz category',  
                   'Knowledge representation', 'implicit vs explicit', 
                   'Business Use Case','Technical Application','Type of Learning', 
                   'Symbolic terms', 'Neural terms']

In [13]:
cat_var_lookup = defaultdict(int)
i = 0

for itm in neural_sup_terms:
  cat_var_lookup[itm]=i
  i+=1

for itm in bool_columns:
  cat_var_lookup[itm]=i
  i+=1

for col in columns:
  terms = []
  for t in df[col].unique():
    _terms = str(t).split('\n')
    for term in _terms:
      term=term.split(',')[0]
      if term != '': terms.append(term)
  for t in set(terms):
    cat_var_lookup[t]=i
    i+=1

In [14]:
# cat_var_lookup_list = sorted(list(cat_var_lookup.items()),key=lambda tup: tup[1])

In [15]:
var_cat_lookup = {v: k for k, v in cat_var_lookup.items()}

In [16]:
def parseValue(s):
  m = re.match(r".*(\((.*)\))",s)
  try:
    s = m.group(2)
  except:
    pass
  return ' '.join(word[0].upper() + word[1:] for word in s.split())

In [17]:
for k,v in var_cat_lookup.items():
  var_cat_lookup[k] = parseValue(v)
# var_cat_lookup

# Code

In [18]:
# generate data structures for sanky by combining columns from df

cols=list(df.columns)

# Sanky layers (these don't have to be Series, it's a leagcy idea)
nesy_category = pd.Series([],dtype = 'object')
kautz_cat = pd.Series([],dtype = 'object')
leverage = pd.Series([],dtype = 'object')
kr = pd.Series([],dtype = 'object')
implicit_explicit = pd.Series([],dtype = 'object')
goals = pd.Series([],dtype = 'object')
biz_case = pd.Series([],dtype = 'object')
tech_app = pd.Series([],dtype = 'object')
learning_type = pd.Series([],dtype = 'object')
symb_terms = pd.Series([],dtype = 'object')
neural_terms = pd.Series([],dtype = 'object')
neural_sup_terms = pd.Series([],dtype = 'object')

for row in df.itertuples():
  nesy_category[len(nesy_category)] = [cat_var_lookup[row[cols.index('NeSy Category')+1]]]
  kautz_cat[len(kautz_cat)] =         [cat_var_lookup[row[cols.index('Kautz category')+1]]]

  #leverage
  _leverage = []
  if row[cols.index('leverages natural language structure')+1]:
    _leverage.append(cat_var_lookup['leverages natural language structure'])
  if row[cols.index('leverages relational structure')+1]:
    _leverage.append(cat_var_lookup['leverages relational structure'])  

  if len(_leverage) == 0:
    _leverage.append(cat_var_lookup['No Structure Leverage'])
  leverage[len(leverage)]=_leverage

  #knowledge representation
  _kr = []
  if type(row[cols.index('Knowledge representation')+1]) == str:
    vals=row[cols.index('Knowledge representation')+1].split('\n')
    for v in vals:
      v=v.split(',')[0]
      if v != '':_kr.append(cat_var_lookup[v])
    kr[len(kr)] = _kr 
  else:kr[len(kr)]=[]
   
  #implicit_explicit
  _implicit_explicit = []
  if type(row[cols.index('implicit vs explicit')+1]) == str:
    vals=row[cols.index('implicit vs explicit')+1].split('\n')
    for v in vals:
      v=v.split(',')[0]
      if v != '':_implicit_explicit.append(cat_var_lookup[v])
    implicit_explicit[len(implicit_explicit)] = _implicit_explicit
  else:implicit_explicit[len(implicit_explicit)]=[]

  #goals
  goalstrings=['reasoning', 'out-of-distribution generalization', 'interpretability', 'reduced data', 'transferability']
  _goals=[]
  for gstr in goalstrings:
    # print(row[cols.index(gstr)+1])
    if row[cols.index(gstr)+1]:
      if v != '':_goals.append(cat_var_lookup[gstr])
  if len(_goals) == 0:
    _goals.append(cat_var_lookup['none'])
  goals[len(goals)] = _goals


  #business use case
  _biz_case=[]
  if type(row[cols.index('Business Use Case')+1]) == str:
    vals=row[cols.index('Business Use Case')+1].split('\n')
    for v in vals:
      v=v.split(',')[0]
      if v != '':_biz_case.append(cat_var_lookup[v])
    biz_case[len(biz_case)] = _biz_case 
  else:biz_case[len(biz_case)]=[]


  #Technical Application
  _tech_app=[]
  if type(row[cols.index('Technical Application')+1]) == str:
    vals=row[cols.index('Technical Application')+1].split('\n')
    for v in vals:
      v=v.split(',')[0]
      if v != '':_tech_app.append(cat_var_lookup[v])
    tech_app[len(tech_app)] = _tech_app
  else:tech_app[len(tech_app)]=[]


  #Learning_type
  _learning_type=[]
  if type(row[cols.index('Type of Learning')+1]) == str:
    vals=row[cols.index('Type of Learning')+1].split('\n')
    for v in vals:
      v=v.split(',')[0]
      if v != '':_learning_type.append(cat_var_lookup[v])
    learning_type[len(learning_type)] = _learning_type
  else:learning_type[len(learning_type)]=[]

  #symb_terms
  _symb_terms=[]
  if type(row[cols.index('Symbolic terms')+1]) == str:
    vals=row[cols.index('Symbolic terms')+1].split('\n')
    for v in vals:
      v=v.split(',')[0]
      if v != '':_symb_terms.append(cat_var_lookup[v])
    symb_terms[len(symb_terms)] = _symb_terms
  else:symb_terms[len(symb_terms)]=[]

  #neural_terms
  _neural_terms=[]
  if type(row[cols.index('Neural terms')+1]) == str:
    vals=row[cols.index('Neural terms')+1].split('\n')
    for v in vals:
      v=v.split(',')[0]
      if v != '': _neural_terms.append(cat_var_lookup[v])
    neural_terms[len(neural_terms)] = _neural_terms
  else:neural_terms[len(neural_terms)]=[]


  # neural_sup_terms
  
  
  neural_sup_terms_dict={
  'Early Generation':       ['neural network (NN)','convolutional neural network (CNN)'],
  'Graphical Models':       ['graph neural network (GNN)','deep belief network (DBN)'],
  'Sequence-to-Sequence':   ['recurrent neural network (RNN)','recursive neural network (RcNN)'],
  'Neuro-Symbolic':         ['logic tensor network (LTN)','recursive neural knowledge network (RNKN)','Tensor Product Representation (TPR)'],
  'Memory Networks':        ['transformer','attention network (Attn)','Memory Network'],
  'Hybrid Models':          ['reinforcement learning (RL)'],
  'Linear Models' :         ['SVM']
  }
  

  
  


  _neural_sup_terms=[]
  for key in neural_sup_terms_dict.keys():
    vals=row[cols.index('Neural terms')+1].split('\n')
    for v in vals:
      v=v.split(',')[0]
      # try:
      if v in neural_sup_terms_dict[key]:
        _neural_sup_terms.append(cat_var_lookup[key])
      # except:
        # pass  
  neural_sup_terms[len(neural_sup_terms)] = _neural_sup_terms



In [19]:
def generateLinks(columns,palette):
  links=[]
  epsilon = 1.0e-10 #prevent division by zero

  
  # flatten_matrix = [val for sublist in matrix for val in sublist]
  source_variables = set([e for el in columns[0] for e in el]) 
  colors_dict = defaultdict()
  for i,c in enumerate(source_variables):
    colors_dict[c] = palette[i]

  for row_idx in range(len(columns[0])):
    for col_idx in range(len(columns)-1):
      
      sources = columns[col_idx][row_idx]

      for source in sources:
        # always use the color from the first column
        if col_idx == 0:
          color = colors_dict[source]

        # 1 / epsilon + (length of sourse * length of target)
        # weight = 1/(epsilon+len(columns[col_idx][row_idx])*len(columns[col_idx+1][row_idx]))
        weight = 1/len(columns[col_idx+1][row_idx])
        targets = columns[col_idx+1][row_idx]

        for target in targets:
          links.append((source, target, weight, color))


  return(np.array(links))

# Choose your layers and make sanky

In [20]:
colors=["#afe979", "#296afb", "#57bcd6", "#450256", "#89b79f", "#9c4397", "#2a91a7", "#ff2f6d", "#005d79"]
list(reversed(colors))

['#005d79',
 '#ff2f6d',
 '#2a91a7',
 '#9c4397',
 '#89b79f',
 '#450256',
 '#57bcd6',
 '#296afb',
 '#afe979']

In [21]:
def makeLabels(st_links):
  labels=[None]*150

  # source labels: set(st_links[:,0:1].flatten())
  for i in set(st_links[:,0:1].flatten()):
    labels[int(i)]=var_cat_lookup[int(i)]


  # target labels: set(st_links[:,1:2].flatten())
  for i in set(st_links[:,1:2].flatten()):
    labels[int(i)]=var_cat_lookup[int(i)]

  return labels

In [22]:

colors=["rgba(32,112,180,.75)","rgba(137,183,159,.75)","rgba(69,2,86,.75)","rgba(156,67,151,.75)","rgba(87,188,214,.75)"]*10
colors=["#450256","#57bcd6","#89b79f","#9c4397","#ff2f6d","#2a91a7","#afe979","#296afb","#005d79"]*10
# colors=["#afe979", "#296afb", "#57bcd6", "#450256", "#89b79f", "#9c4397", "#2a91a7", "#ff2f6d", "#005d79"]
# colors=list(reversed(colors))*10


In [23]:
# Choose from below sanky layers:
layers=['nesy_category','leverage','kr','implicit_explicit','goals','biz_case','tech_app','learning_type','symb_terms','neural_terms','neural_sup_terms','kautz_cat']
layers

['nesy_category',
 'leverage',
 'kr',
 'implicit_explicit',
 'goals',
 'biz_case',
 'tech_app',
 'learning_type',
 'symb_terms',
 'neural_terms',
 'neural_sup_terms',
 'kautz_cat']

In [24]:
import plotly.graph_objects as go

In [25]:
st_links = generateLinks([leverage,tech_app],colors)
labels=makeLabels(st_links)

weights = [1]*len(st_links[:,2:3].flatten())
weights = st_links[:,2:3].flatten()

# data to dict, dict to sankey
link = dict(source = st_links[:,0:1].flatten(), target = st_links[:,1:2].flatten(), value = weights, color = st_links[:,3:4].flatten())
node = dict(label = labels, pad=25, thickness=10, color="#333333")
data = go.Sankey(link = link, node=node)
# plot

fig = go.Figure(data)
fig.update_layout(
    hovermode = 'x',
    title = "",
    font=dict(size = 13, color = '#333333'),
    height = 500,
    width = 600,
  font_family="Arial"
  )
fig.show()

In [26]:
fig.write_image('kr-neural_sup_terms.svg')